# Mount your google drive

You can mount your google drive in colab in order to have access to permanent data storage.

In [1]:
import os
# mount your google drive
# you will need to authorize the access
from google.colab import drive
root = '/content/gdrive/'
drive.mount(root)

# this is the root file for your google drive
gdrive_root = os.path.join(root, 'My Drive')

Mounted at /content/gdrive/


In [2]:
# now, you can access and explore the contents of your gdrive:
gdrive_files = os.listdir(gdrive_root)
print(gdrive_files)

['Copy of CV.gdoc', 'Einkaufsabbrechnung Vangerow Nov 15 Jun 16.gsheet', 'Janelia Visit June 16.gdoc', 'presentationDavisVisit.gslides', 'To-do list.gsheet', 'cremi - cantor train   test.gsheet', "Niko's hat.gdoc", 'cremi_defects.gsheet', 'cremi_results.gsheet', 'cantor_comparisson.gsheet', 'Janelia Visitor Projects.gdoc', 'results_large_blocks.gsheet', 'Validation Sample D.gsheet', 'Processing Sample D.gslides', 'Student Topics.gdoc', 'Paper 13paper.pdf', 'ilastik benchmarks.gsheet', 'summary: defect handling.gslides', 'sampleD_segmentation', 'Compute Resources.gdoc', 'own work + nblast.gslides', 'Research Overview - Janelia Visit 03 17.gslides', 'Grayscale DT + WS.gslides', 'vec_images', 'mala+mc lmc', 'poster_bkp.gslides', 'cube.jpg', 'mscs', 'debeka', 'test_data.zip', 'mc_mp', 'Copy of own work + nblast.gslides', 'Einkaufsabbrechnung Vangerow Dez 16.gsheet', 'Einkaufsabbrechnung Vangerow Mai 17.gsheet', 'owt:16_05_17.gslides', 'Comparsion MALA + MC LMC.gslides', 'execution_timings.

In [3]:
# make a folder to store results for this course
res_folder = os.path.join(gdrive_root, 'dl-course')
os.makedirs(res_folder, exist_ok=True)